# INFS7410 Week 3 Practical

##### version 1.3

###### The INFS7410 Teaching Team

---

##### About today's tutorial

This week is an introduction to ranking functions (also often called retrieval models, ambiguously), which are used to score and order documents in search engines. You will learn about simple, term-based ranking functions this week -- and we will examine more complex ranking functions as we go ahead in the course.

##### Tutorial Etiquette
Please refrain from loud noises, irrelevant conversations and use of mobile phones during tutorial activities. Be respectful of everyone's opinions and ideas during the tutorial activities. You will be asked to leave if you disturb. Remember the tutor is there to help you understand and learn, not to provide debugging of your code or solutions to assignments. 

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/nMyg3XzGx_w" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Exercise 0: Re-indexing the collection

Here, we need to re-index the msmarco collection with a few more attributes, namely the positions and document vectors. You have two options to get this new index (you only need to do one of the two options below). The first option is to run the script below to re-index the collection files from week 1:

Option 1: re-index using pyserini

In [5]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 9 -input collection/collection_jsonl \
 -index indexes/lucene-index-msmarco-passage-vectors-noProcessing \
 -optimize \
 -storeRaw \
 -stemmer none \
 -keepStopwords \
 -storePositions \
 -storeDocvectors 

2021-08-17 23:03:42,856 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2021-08-17 23:03:42,861 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2021-08-17 23:03:42,862 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2021-08-17 23:03:42,863 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: collection/collection_jsonl
2021-08-17 23:03:42,863 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2021-08-17 23:03:42,863 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2021-08-17 23:03:42,864 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 9
2021-08-17 23:03:42,864 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Stemmer: none
2021-08-17 23:03:42,864 INFO  [main] index.IndexColle

Exception in thread "pool-2-thread-3" java.lang.OutOfMemoryError: Java heap space
	at org.apache.lucene.util.ArrayUtil.growExact(ArrayUtil.java:302)
	at org.apache.lucene.util.ArrayUtil.grow(ArrayUtil.java:311)
	at org.apache.lucene.util.BytesRefHash$DirectBytesStartArray.grow(BytesRefHash.java:563)
	at org.apache.lucene.util.BytesRefHash.add(BytesRefHash.java:278)
	at org.apache.lucene.index.SortedDocValuesWriter.addOneValue(SortedDocValuesWriter.java:88)
	at org.apache.lucene.index.SortedDocValuesWriter.addValue(SortedDocValuesWriter.java:76)
	at org.apache.lucene.index.DefaultIndexingChain.indexDocValue(DefaultIndexingChain.java:614)
	at org.apache.lucene.index.DefaultIndexingChain.processField(DefaultIndexingChain.java:479)
	at org.apache.lucene.index.DefaultIndexingChain.processDocument(DefaultIndexingChain.java:406)
	at org.apache.lucene.index.DocumentsWriterPerThread.updateDocument(DocumentsWriterPerThread.java:250)
	at org.apache.lucene.index.DocumentsWriter.updateDocument(Docu

Option 2: download the index from this url and unzip it into your indexes folder

https://infs7410.uqcloud.net/lucene-index-msmarco-passage-vectors-noProcessing.zip

---

Now that we have the new index, we are going to set up all the libraries and functions to use in the rest of this prac.

In [56]:
from pyserini.search import SimpleSearcher
from pyserini.analysis import Analyzer, get_lucene_analyzer
from pyserini.index import IndexReader
from collections import Counter

lucene_analyzer = get_lucene_analyzer(stemming=False, stopwords=False)
analyzer = Analyzer(lucene_analyzer)
searcher = SimpleSearcher('indexes/lucene-index-msmarco-passage-vectors-noProcessing/')
searcher.set_analyzer(lucene_analyzer)

index_reader = IndexReader('indexes/lucene-index-msmarco-passage-vectors-noProcessing/')

queries = []
with open("queries.tsv", "r") as f:
    for line in f.readlines():
        parts = line.split("\t")
        # parts[0] ~> topic id
        # parts[1] ~> query
        queries.append((parts[0], parts[1].strip()))

# You should remember these funcations from last week.
# The search method has been slightly modified to take in a scorer function.
def search(run_file: str, k: int=10, scorer=None):
    assert scorer is not None
    with open(run_file, "w") as f:
        for topic_id, query in queries: # topic id is important here.
            hits = searcher.search(query, k=k)
            q_terms = analyzer.analyze(query)
            # print(q_terms)
            run = []
            for i, hit in enumerate(hits):
                # Compute the statistics.
                tf = index_reader.get_document_vector(hit.docid)
                df= {term: (index_reader.get_term_counts(term, analyzer=None))[0] for term in tf.keys()}
                
                doc_len = len(tf)
                score = 0
                
                # Critically, we only include terms that appear in the query and document.
                c = list((Counter(q_terms) & Counter(tf.keys())).elements())
                # print(c)
                for term in c:
                    # Call the scoring function (you will implement these below).
                    score += scorer(tf[term], df[term], doc_len)
                run.append((hit.docid, score))
            
            # Sort the run by the score.
            for i, r in enumerate(sorted(run, key=lambda x: x[1], reverse=True)):
                # Write the results to our file.
                f.write(f"{topic_id} Q0 {r[0]} {i} {r[1]} infs7410_w3\n")
                

def print_line(measure, scope, value): 
    print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

## Exercise 1: Introduction to ranking functions

All ranking functions can be defined as the score between a document ($D$) and a query ($Q$): $\text{score}(D,Q)$. In this exercise you will learn the intuitions behind the most basic ranking functions. You will then be presented with the formalisation of a typically robust and effective ranking function, called BM25. You will implement these ranking functions in Python below. First, however, let's look at some basic ranking functions and the intuitions behind them. 

### Term Frequency 

Term frequency (tf) is a very simple ranking function which ranks documents by the sum of the number of times each term in a query appears in a document. Formally, we can express this as:
$$
score(D,Q)=\sum_{i=1}^{|Q|}f(q_i,D)
$$
Where $f(q_i, D)$ is a function which outputs the count of the term $q_i$ in document $D$ (thus, $0$ if the query term is not in the document), and |Q| is the size of the query. Effectivelly, the summation is over all the terms in the query. This should sound familiar: it should resonate with the idea from the abstract model of retrieval and ranking we mentioned in week 3 to introduce inverted index. In there, we said we would have only considered for scoring the terms that are in the query, not all terms that are in the document (and thta motivated the structure of the inverted index).

Let's turn to implementing this function. We have provided a skeleton function below where you can write your implementation of TF.

In [57]:
def tf(tf, df, doc_len):
    # TODO: Your implementation here
    return tf

search("week3-tf.run", k=10, scorer=tf)

**QUESTION:** _In your opinion, is this a good ranking function? What situations would cause this ranking function to output a high score or a low score? Are these situations clear indication that a document is highly relevant, and unlikely relevant, respectively?_ 

### Inverse Document Frequency

Inverse document frequency (idf) is another simple ranking function which measures the amount of information a term provides, or, generally speaking, the importance of a term in the collection. In other words, it is the number of times a term appears in a document with respect to the collection as a whole. Formally, we can express this as:
$$
score(D, Q) = \sum\limits_{i=1}^{|Q|} log\frac{N}{1 + n_t}
$$
Where $N$ is the total number of documents in the collection, and $n_t$ is the number of documents the term appears in. The ratio is logarithmically scaled for smoothing.

Note: idf only depends on the term, not of the document in which the term appears in, that is, the idf of a term is the same, regardless of the document you are considering.

this time, you will be implementing idf.

In [59]:
N = index_reader.stats()["documents"]

import math

def idf(tf, df, doc_len):
    # TODO: Your implementation here
    # Note: Use the variable above to help you.
    
    # print(math.log(N/(1+df)))
    return math.log(N/(1+df))


search("week3-idf.run", k=1000, scorer=idf)

2.9150364071872197
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
2.9150364071872197
8.755071043752038
4.966667657676433

2.9150364071872197
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
2.9150364071872197
8.755071043752038
2.9150364071872197
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
4.966667657676433
8.755071043752038
4.966667657676433
8.755071043752038
2.9150364071872197
8.755071043752038
2.9150364071872197
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755

8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
2.9150364071872197
4.966667657676433
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
2.9150364071872197
4.966667657676433
2.9150364071872197
8.755071043752038
2.9150364071872197
8.755071043752038
2.9150364071872197
8.755071043752038
2.9150364071872197
8.755071043752038
2.9150364071872197
8.755071043752038
2.9150364071872197
8.755071043752038
2.9150364071872197
8.755071043752038
8.755071043752038
8.755071043752038
8.755071043752038
8

2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2.9150364071872197
4.966667657676433
2

2.791429002243355
7.588518565640691
5.740366374640485
0.6117410739279476
7.588518565640691
7.5642401716546575
7.588518565640691
7.5642401716546575
2.791429002243355
0.6117410739279476
7.5642401716546575
7.588518565640691
7.5642401716546575
7.588518565640691
7.5642401716546575
2.791429002243355
0.6117410739279476
7.588518565640691
0.6117410739279476
7.588518565640691
5.740366374640485
0.6117410739279476
7.588518565640691
7.5642401716546575
2.791429002243355
0.6117410739279476
7.588518565640691
7.588518565640691
7.5642401716546575
7.588518565640691
7.5642401716546575
7.588518565640691
7.5642401716546575
7.588518565640691
7.5642401716546575
2.791429002243355
0.6117410739279476
7.5642401716546575
7.588518565640691
7.5642401716546575
7.588518565640691
7.5642401716546575
7.588518565640691
7.5642401716546575
7.588518565640691
7.5642401716546575
7.588518565640691
7.5642401716546575
0.6117410739279476
7.588518565640691
7.5642401716546575
0.6117410739279476
7.588518565640691
7.5642401716546575
0

...

12.776127810204306
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
5.269118534879157
0.4064199484562767
0.3183079855989159
9.709005540563643
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
0.4064199484562767
9.709005540563643
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.3183079855989159
5.269118534879157
0.3183079855989159
9.709005540563643
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
0.3183079855989159
9.709005540563643
4.762890559618438
3.054944214617011
5.269

4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.318307

4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.0549

4.762890559618438
0.4064199484562767
0.3183079855989159
5.269118534879157
3.054944214617011
0.4064199484562767
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
0.4064199484562767
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.7

4.762890559618438
3.054944214617011
0.3183079855989159
3.054944214617011
0.4064199484562767
0.3183079855989159
5.269118534879157
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
9.709005540563643
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
0.4064199484562767
0.3183079855989159
5.269118534879157
0.4064199484562767
0.3183079855989159
9.709005540563643
4.762890559618438
3.054944214617011
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.762890559618438
3.054944214617011
0.4064199484562767
0.3183079855989159
4.

**QUESTION:** _In your opinion, is this a good ranking function? What situations would cause this ranking function to output a high score or a low score? Are these situations clear indication that a document is highly relevant, and unlikely relevant, respectively?_

### TF-IDF

Term Frequency-Inverse Document Frequency (tf-idf) is the combination of the two previous ranking functions. It can be expressed formally as:
$$
score(D, Q) = \sum\limits_{i=1}^{|Q|} tf(D, i) * idf(i)
$$
Note: we wrote $\text{idf}(D,Q)$ to indicate that we are using idf to score document D, but this is more correctly written as $\text{idf}(Q)$ (because idf does not depend on D, or at least the version of idf that we have discussed).

In the cell below, write your implementation of tf-idf.

In [60]:
def tfidf(tf, df, doc_len):
    # TODO: Your implementation here
    return (tf*math.log(N/(1+df)))

search("week3-tfidf.run", k=1000, scorer=tfidf)

**QUESTION:** _How does this ranking function compare to the previous two ranking functions you have observed? Do you think it is better than the above ranking functions? Why?_

**QUESTION:** _*In your opinion, is this a good ranking function? What situations would cause this ranking function to output a high score or a low score? Are these situations clear indication that a document is highly relevant, and unlikely relevant, respectively?*_

### BM25

You may have seen BM25 used in previous tutorials, when we asked you to create retrieval runs. In most scenarios, BM25 outperforms tf-idf -- in fact, BM25 is the de-facto standard retrieval model a number of IR toolkits ship with. For example Lucene and Elasticsearch, which are popular with companies that want to deploy a search service, have both BM25 set as their default ranking method. 

BM25 is formally defined as:
$$
\text{score}(D,Q)=\sum^{|Q|}_{i=1}\text{idf}(q_i)\cdot\frac{f(q_i,D)\cdot(k_1+1)}{f(q_i,D)+k_1\cdot(1-b+b\cdot\frac{|D|}{avgdl})}
$$

Where $f(q_i,D)$ is tf, $|D|$ is the length of the document in number of terms, $avgdl$ is the average document length in the collection (i.e. $\sum_{j=1}^{|C|} |D_j|/|C|$, where $|C|$ is the number of documents in the collection), and $k_1$ and $b$ are parameters (which you may want to tune). Typical values for the these parameters are $k=1.2;b=0.75$ -- but note that these values were empirically derived on early TREC collections, i.e. mainly for ad-hoc retrieval on newswire collections.

Back to the practical exercise. Implement your version of BM25 below. Note that, unlike tf-idf, BM25 is a more complex formula: you may want to create some tests to ensure you have correctly implemented the formula. 

In [62]:
avg_dl = index_reader.stats()["total_terms"]/index_reader.stats()["documents"]
k_1 = 1.2
b = 0.75
def bm25(tf, df, doc_len):
    # TODO: Your implementation here
    # Note: Use the variables above to help you.
    idf = math.log(N/(1+df))
    
    var_a = tf*(k_1 + 1)
    var_b = tf+k_1*(1 - b + (b*doc_len/avg_dl))
    
    return idf*var_a/var_b
    

search("week3-bm25.run", k=1000, scorer=bm25)

**QUESTION:** _How does this ranking function compare to the previous three ranking functions you have observed? Do you think it is better than the above ranking functions?_

**QUESTION:** _*In your opinion, is this a good ranking function? What situations would cause this ranking function to output a high score or a low score? Are these situations clear indication that a document is highly relevant, and unlikely relevant, respectively?*_

## Exercise 2: Comparing ranking functions

Use the results files produced by each of the ranking functions to complete the table below and create appropriate visualisations using the plotting scripts from last week (and possible extensions to those scripts you may have implemented). Also, include the results from the previous week in this table (Pyserini BM25). Make sure you use the run where k=1000.

Note that in this exercise, only the first 1,000 documents are returned. This means that your precision and recall are measured at cutoff depth 1000 (P@1000 and R@1000). Since this exercise focuses on ranking functions and nothing about the documents or queries are changing, the overall (set-)precision and (set-)recall of the runs would not change. We therefore measure precision and recall only for the top-1000 documents (thus considering if the ranking functions leave out some relevant documents from the top 1,000 ranks). Note again that you do not have to do anything special, we tell you this in case you were wondering why the precision and recall values are different between runs.

|                | Precision | Recall | MAP  | nDCG |
| -------------- | --------- | ------ | ---- | ---- |
| Your tf-idf    |           |        |      |      |
| Your BM25      |           |        |      |      |
| Pyserini BM25  |           |        |      |      |


**QUESTION:** _Which run was the most effective? Describe the results of your table above by making comparisons between each run. Make comments about statistical significance once you have completed that._

**QUESTION:** _Which results visualisation(s) did you choose? Why did you choose this/these visualisation(s)? What is this/are these visualisation(s) telling you?_

## Take-home exercise

If you complete this tutorial in your allocated time, you may want to create for yourself a script to produce results tables like the one in this exercise. You can do this in any programming language you like, and it is outside the scope of this subject (so the teaching team will not help you and it will not be marked). You will, however, be creating a lot of results tables in this subject, so it will make your life easier -- for example it will cut some of the time required for the construction and reporting of the results for the project.